## SETUP
---
Instalação de bibliotecas e importação de dados

Importação de dados do GitHub:

In [ ]:
%pip install -q gitpython

import os
import shutil
import git
import tempfile
from pathlib import Path

REPO_URL = "https://github.com/GTazz/CS-Graduation.git"
FOLDER_NAME = "2-Semestre/3 - SO/TrabalhoArquivos/prontuarios"

destination_dir = Path(os.getcwd()) / Path(FOLDER_NAME).name

if not os.path.exists(destination_dir):
    with tempfile.TemporaryDirectory() as tmp:
        git.Repo.clone_from(REPO_URL, tmp, depth=1)

        target_dir = Path(tmp) / FOLDER_NAME
        
        shutil.copytree(target_dir, destination_dir, dirs_exist_ok=True)
            
        print("Arquivos do repositório mesclados no diretório atual.")

Criação de função de geração de árvore:

In [ ]:
%pip install -q matplotlib

import os
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
from PIL import Image

# ---- Helpers to build and layout the tree (width-aware, no overlap) ----
def build_fs_tree(root, base_path=None):
    if base_path is None:
        base_path = root
    name = os.path.basename(os.path.normpath(root)) or root
    node = {"name": name, "is_dir": True, "children": [], "full_path": root}
    try:
        entries = sorted(os.listdir(root))
    except FileNotFoundError:
        raise FileNotFoundError(f"Caminho não encontrado: {root}")
    for item in entries:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            node["children"].append(build_fs_tree(path, base_path))
        else:
            node["children"].append({"name": item, "is_dir": False, "children": [], "full_path": path})
    return node

def annotate_sizes(node, char_w=0.14, base_w=0.8, min_w=1.3, box_h=0.8, sep=0.6):
    """Annotate each node with box_w/box_h and subtree_w using children widths."""
    label = node["name"] + ("/" if node.get("is_dir", False) else "")
    node["label"] = label

    # Check if it's a JPEG file
    node["is_image"] = node["name"].lower().endswith((".jpeg", ".jpg"))

    # Increase box height for images to accommodate image + caption
    actual_box_h = box_h * 2.0 if node["is_image"] else box_h

    node["box_w"] = max(min_w, base_w + char_w * len(label))
    node["box_h"] = actual_box_h
    for c in node.get("children", []):
        annotate_sizes(c, char_w, base_w, min_w, box_h, sep)
    if node.get("children"):
        total_children = sum(c["subtree_w"] for c in node["children"]) + sep * (len(node["children"]) - 1)
        node["subtree_w"] = max(node["box_w"], total_children)
    else:
        node["subtree_w"] = node["box_w"]
    return node

def assign_positions(node, x_left=0.0, level=0, y_spacing=2.2, sep=0.6):
    """Assign x,y positions so sibling subtrees are spaced by subtree width (no overlap)."""
    node["y"] = -level * y_spacing
    if not node.get("children"):
        node["x"] = x_left + node["box_w"] / 2.0
        return node["x"]
    cursor = x_left
    for i, c in enumerate(node["children"]):
        assign_positions(c, cursor, level + 1, y_spacing, sep)
        cursor += c["subtree_w"] + (sep if i < len(node["children"]) - 1 else 0.0)
    span_left = x_left
    span_right = x_left + sum(c["subtree_w"] for c in node["children"]) + sep * (len(node["children"]) - 1)
    node["x"] = (span_left + span_right) / 2.0
    return node["x"]

def count_depth(node):
    if not node.get("children"):
        return 1
    return 1 + max(count_depth(c) for c in node["children"])

def find_upwards(start_dir, name):
    p = os.path.abspath(start_dir)
    while True:
        candidate = os.path.join(p, name)
        if os.path.exists(candidate):
            return candidate
        parent = os.path.dirname(p)
        if parent == p:
            return None
        p = parent

def resolve_root(root_name):
    # 1. If path exists directly (relative or absolute)
    if os.path.exists(root_name):
        return os.path.abspath(root_name)
    # 2. Try inside a data/ folder upward
    data_base = find_upwards(os.getcwd(), "data")
    if data_base:
        candidate = os.path.join(data_base, root_name)
        if os.path.exists(candidate):
            return candidate
    # 3. Try upward search for folder name itself
    return find_upwards(os.getcwd(), root_name)


def _content_areas(x, y, w, h, padding_frac=0.08, image_frac=0.70):
    pad_x = w * padding_frac
    pad_y = h * padding_frac
    left = x - w/2 + pad_x
    right = x + w/2 - pad_x
    top = y + h/2 - pad_y
    bottom = y - h/2 + pad_y
    content_w = right - left
    content_h = top - bottom
    img_area_h = content_h * image_frac
    txt_area_h = content_h - img_area_h
    return left, right, top, bottom, content_w, content_h, img_area_h, txt_area_h


def _draw_image_inside_box(ax, img, box_patch, x, y, w, h, padding_frac=0.08, image_frac=0.70):
    # Compute inner areas
    left, right, top, bottom, content_w, content_h, img_area_h, txt_area_h = _content_areas(x, y, w, h, padding_frac, image_frac)
    # Preserve aspect ratio to fit into (content_w x img_area_h)
    aspect = img.height / img.width if img.width else 1.0
    w_by_width = content_w
    h_by_width = w_by_width * aspect
    if h_by_width <= img_area_h:
        img_w = w_by_width
        img_h = h_by_width
    else:
        img_h = img_area_h
        img_w = img_h / aspect
    img_xmin = x - img_w/2
    img_xmax = x + img_w/2
    img_ymax = top
    img_ymin = img_ymax - img_h
    ax.imshow(img, extent=[img_xmin, img_xmax, img_ymin, img_ymax], zorder=3, clip_path=box_patch, clip_on=True)
    caption_center_y = bottom + txt_area_h / 2
    return caption_center_y


def draw_tree_auto(root_name):
    base = resolve_root(root_name)
    if base is None:
        print(f"Pasta '{root_name}' não encontrada a partir de {os.getcwd()}.")
        return
    tree = build_fs_tree(base)
    tree = annotate_sizes(tree)
    assign_positions(tree)
    depth = count_depth(tree)
    fig_w = max(10, tree["subtree_w"] * 0.9 + 2)
    fig_h = max(4, depth * 1.7)
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))

    # Edges
    stack = [tree]
    while stack:
        n = stack.pop()
        for c in n.get("children", []):
            ax.plot([n["x"], c["x"]], [n["y"] - 0.35, c["y"] + 0.35], color="#4e555b", linewidth=1.2, zorder=1)
            stack.append(c)

    # Nodes (boxes)
    stack = [tree]
    while stack:
        n = stack.pop()
        x, y = n["x"], n["y"]
        w, h = n["box_w"], n["box_h"]
        face = "#1976d2" if n.get("is_dir", False) else "#9e9e9e"
        box = FancyBboxPatch((x - w/2, y - h/2), w, h,
                             boxstyle="round,pad=0.02,rounding_size=0.08",
                             linewidth=1.0, edgecolor="#2f3941", facecolor=face,
                             alpha=0.95, zorder=2)
        ax.add_patch(box)

        if n.get("is_image", False):
            try:
                img_path = n.get("full_path")
                img = Image.open(img_path).convert("RGBA")
                caption_y = _draw_image_inside_box(ax, img, box, x, y, w, h,
                                                    padding_frac=0.08, image_frac=0.70)
                ax.text(x, caption_y, n["label"], color="white", ha="center", va="center",
                        fontsize=9, family="monospace", zorder=3, clip_path=box, clip_on=True)
            except Exception:
                # Compute areas and place caption at bottom even on failure; draw placeholder in image area
                left, right, top, bottom, content_w, content_h, img_area_h, txt_area_h = _content_areas(x, y, w, h, 0.08, 0.70)
                icon_y = top - img_area_h/2
                ax.text(x, icon_y, "🖼️", color="white", ha="center", va="center",
                        fontsize=14, zorder=3, clip_path=box, clip_on=True)
                caption_y = bottom + txt_area_h/2
                ax.text(x, caption_y, n["label"], color="white", ha="center", va="center",
                        fontsize=9, family="monospace", zorder=3, clip_path=box, clip_on=True)
        else:
            # Normal text centered
            ax.text(x, y, n["label"], color="white", ha="center", va="center",
                    fontsize=10, family="monospace", zorder=3, clip_path=box, clip_on=True)

        stack.extend(n.get("children", []))

    ax.set_xlim(-0.5, tree["subtree_w"] + 0.5)
    ax.set_ylim(-depth * 2.2 + 0.8, 0.8)
    ax.set_title(f"Estrutura em árvore – diretório '{os.path.basename(base)}'", fontsize=14, weight="bold")
    ax.axis("off")
    ax.margins(x=0.02, y=0.02)
    plt.tight_layout()
    plt.show()

# # ---- Run ----
# draw_tree_auto("prontuarios")

Criação de função de geração de tabela e gráfico:

In [ ]:
%pip install -q pandas jinja2

import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt

def list_files_as_dataframe(root_path):
    """
    Percorre recursivamente um diretório e retorna um DataFrame categorizado com informações dos arquivos.
    
    Retorna DataFrame com colunas organizadas por hierarquia:
    - Status Paciente: Ativo ou Arquivado
    - Nome Paciente: ID e nome do paciente
    - Categoria: Tipo de documento (Perfil, Consultas, Exames, Tratamentos)
    - Nome: nome do arquivo/diretório
    - Tipo: 'Diretório' ou 'Arquivo'
    - Extensão: extensão do arquivo
    - Tamanho (KB): tamanho do arquivo em KB
    - Caminho Completo: caminho relativo completo
    """
    base = resolve_root(root_path)
    if base is None:
        print(f"Pasta '{root_path}' não encontrada.")
        return None
    
    data = []
    base_path = Path(base)
    
    # Percorrer recursivamente
    for item in sorted(base_path.rglob('*')):
        rel_path = item.relative_to(base_path)
        parts = rel_path.parts
        
        # Determinar categorias baseado na estrutura de pastas
        status_paciente = ''
        nome_paciente = ''
        categoria = ''
        
        if len(parts) > 0:
            # Primeiro nível: pacientes_ativos ou pacientes_arquivados
            if 'pacientes_ativos' in parts[0]:
                status_paciente = 'Ativo'
            elif 'pacientes_arquivados' in parts[0]:
                status_paciente = 'Arquivado'
            
            # Segundo nível: nome do paciente (ex: P0-Olavo_de_Carvalho)
            if len(parts) > 1:
                nome_paciente = parts[1]
                
                # Terceiro nível: categoria do documento
                if len(parts) > 2:
                    categoria_raw = parts[2]
                    # Mapear para nomes mais legíveis
                    categoria_map = {
                        'consultas': 'Consultas',
                        'exames': 'Exames',
                        'tratamentos': 'Tratamentos',
                        'cirurgias': 'Cirurgias'
                    }
                    categoria = categoria_map.get(categoria_raw, categoria_raw.capitalize())
                elif item.is_file():
                    # Arquivo diretamente na pasta do paciente (ex: profile.jpg)
                    categoria = 'Perfil'
        
        if item.is_dir():
            data.append({
                'Status Paciente': status_paciente,
                'Nome Paciente': nome_paciente,
                'Categoria': categoria,
                'Nome': item.name,
                'Tipo': 'Diretório',
                'Extensão': '',
                'Tamanho (KB)': '',
                'Caminho Completo': str(rel_path)
            })
        else:
            size = item.stat().st_size
            size_kb = round(size / 1024, 2) if size > 0 else 0
            ext = item.suffix
            data.append({
                'Status Paciente': status_paciente,
                'Nome Paciente': nome_paciente,
                'Categoria': categoria,
                'Nome': item.name,
                'Tipo': 'Arquivo',
                'Extensão': ext,
                'Tamanho (KB)': size_kb,
                'Caminho Completo': str(rel_path)
            })
    
    df = pd.DataFrame(data)
    return df

def save_and_display_csv(root_path, csv_filename='arquivos_listagem.csv'):
    """
    Gera DataFrame categorizado, salva como CSV e exibe no notebook com agrupamento.
    """
    df = list_files_as_dataframe(root_path)
    if df is None:
        return
    
    # Ordenar por categorias lógicas
    df = df.sort_values(
        by=['Status Paciente', 'Nome Paciente', 'Categoria', 'Tipo', 'Nome'],
        ascending=[True, True, True, False, True]  # Diretórios antes de Arquivos
    ).reset_index(drop=True)
    
    # Salvar como CSV
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"✓ Arquivo CSV salvo: {csv_filename}")
    
    # Criar estilo neutro e profissional
    styled = df.style.set_properties(**{
        'text-align': 'left',
        'font-size': '10pt',
        'border': '1px solid #ddd',
        'padding': '8px',
        'color': '#333333'
    }).set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#f5f5f5'), 
            ('color', '#333333'), 
            ('font-weight', 'bold'), 
            ('text-align', 'left'),
            ('padding', '10px'),
            ('border', '1px solid #ddd'),
            ('position', 'sticky'),
            ('top', '0')
        ]},
        {'selector': 'tr:nth-of-type(even)', 'props': [('background-color', '#fafafa')]},
        {'selector': 'tr:nth-of-type(odd)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'tr:hover', 'props': [('background-color', '#f0f0f0')]},
        {'selector': 'td', 'props': [('border', '1px solid #e0e0e0')]}
    ]).hide(axis='index')
    
    # Destacar diretórios vs arquivos com cores neutras
    def color_by_type(val):
        if val == 'Diretório':
            return 'background-color: #e8e8e8; font-weight: bold; color: #333333'
        elif val == 'Arquivo':
            return 'background-color: #ffffff; color: #333333'
        return ''
    
    styled = styled.map(color_by_type, subset=['Tipo'])
    
    return styled

def gerar_relatorio_contagem_categoria(csv_path='mapeamento_prontuarios.csv', mostrar_grafico=True, incluir_perfil=False):
    """Gera contagem de arquivos por categoria a partir do CSV.
    Retorna uma Series com as contagens ordenadas.
    
    Parâmetros:
      csv_path: caminho do CSV previamente gerado (por save_and_display_csv)
      mostrar_grafico: se True, exibe gráfico de barras com as contagens
      incluir_perfil: se False (padrão), exclui categoria 'Perfil' da contagem/gráfico
    """
    if not os.path.exists(csv_path):
        print(f'❌ Arquivo CSV não encontrado: {csv_path}')
        print('Execute primeiro save_and_display_csv() para gerar o arquivo.')
        return None
    
    df = pd.read_csv(csv_path)

    if 'Tipo' in df.columns:
        df_files = df[df['Tipo'] == 'Arquivo'].copy()
    else:
        df_files = df.copy()

    if 'Categoria' not in df_files.columns:
        print("Coluna 'Categoria' não encontrada.")
        return None

    df_files['Categoria'] = df_files['Categoria'].fillna('').replace('', 'Sem categoria')

    # Excluir 'Perfil' ou 'Profile' por padrão (case-insensitive)
    if not incluir_perfil:
        df_files = df_files[~df_files['Categoria'].str.lower().isin(['perfil', 'profile'])]

    counts = (
        df_files.groupby('Categoria')
        .size()
        .sort_values(ascending=False)
    )

    if counts.empty:
        print('Nenhuma categoria com arquivos encontrada.')
        return counts

    if mostrar_grafico:
        plt.figure(figsize=(8, 4))
        counts.plot(kind='bar', color='#1976d2', edgecolor='#2f3941')
        plt.title('Quantidade de arquivos por categoria')
        plt.ylabel('Nº de arquivos')
        plt.xlabel('Categoria')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    return counts


# </> Abstração em Gerência de Arquivos
---
### Exemplo aplicado: Prontuários Hospitalares

### Autores
| Nome | RGM |
|---|---|
| Camilla Macedo Alves | 43713548 |
| Gabriel Oliveira de Souza | 43319327 |
| João Gabriel Rocha Cerqueira | 43912672 |
| Virgínia Mayumi Furushima de Freitas | 42114331 |

**Tema:** Abstração em Gerência de Arquivos
<br/>
**Aplicação:** Prontuários Hospitalares

**Data de Apresentação:** 13/Nov/2025

**Curso:** Ciência da Computação
<br/>
**Disciplina:** Sistemas Operacionais



### Objetivo da Apresentação

Apresentar cada um dos níveis de funcionamento do sistema através das seguintes flags:

- 🔴 **Baixo Nível:** Revisar operações que ocorrem por baixo do SO.
- 🟡 **Abstração SO:** Explicar abstrações de sistemas feitos pelo Sistema Operacional.
- 🟢 **Abstração Programática:** Apresentar uma segunda camada de abstração por cima do SO através da programação.

### Tópicos Chave
0. Abstração e o Papel do Sistema Operacional
1. Estrutura e Diretórios
2. Permissões
3. Mecanismos de Acesso
4. Armazenamento

Caso prático: Prontuários Hospitalares

---

## 0 - Abstração e o Papel do Sistema Operacional

Abstração é o princípio de esconder detalhes de implementação para expor apenas conceitos e operações essenciais. Em Sistemas Operacionais, isso significa transformar hardware (discos, memória, CPU, dispositivos) e mecanismos de baixo nível (blocos, setores, interrupções) em modelos simples de usar: processos, arquivos, diretórios, permissões e chamadas de sistema.

Nesta apresentação, usamos a ideia de camadas:
- Hardware e detalhes físicos ficam “embaixo dos panos”.
- O SO oferece interfaces estáveis (arquivos, pastas, permissões, syscalls) que padronizam o acesso.
- Em cima disso, nosso trabalho adiciona MAIS UMA camada de abstração prática para o domínio de prontuários hospitalares: visualização em árvore, mapeamento completo em CSV e operações de consulta/relatório.

Objetivo do trabalho: tornar tarefas comuns (navegar, buscar, contar, auditar) mais seguras e repetíveis, sem depender de conhecimento de detalhes de baixo nível, alinhando a prática com os conceitos de abstração ensinados em Sistemas Operacionais.

[placeholder][placeholder][placeholder][placeholder]

## 1 - Estrutura e Diretórios Representados em Árvore

### 🔴 Baixo Nível

[placeholder]

### 🟡 Abstração do SO

[placeholder]

### 🟢 Abstração Programática (projeto)

##### Representação de cores
- **🟦 Azul:** Diretório / Pasta
- **⬛ Cinza:** Arquivo

#### 1º Nível Hierárquico

Estrutura completa no diretório raiz `prontuarios/`:

In [ ]:
draw_tree_auto("prontuarios/")

A Hierarquia Nº1 é composta por duas ramificações a partir do diretório raiz `prontuarios/`, sendo elas:

- `pacientes_arquivados/`

- `pacientes_ativos/`

#### 2º Nível Hierárquico

Estrutura no diretório `pacientes_arquivados/`:

In [ ]:
draw_tree_auto("prontuarios/pacientes_arquivados/")

Estrutura no diretório `pacientes_ativos/`:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos/")

A Hierarquia Nº2 é composta por uma ramificação para cada paciente **ativo** ou **arquivado**.

O nome do diretório é composto pelo índice e nome do paciente, no seguinte formato:
``` js
"P" + index + "-" + name
```

#### 3º Nível Hierárquico

Estrutura no diretório de um paciente:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos/P3-Maria_Hipotetica/")

A Hierarquia Nº3 é composta por dois tipos de elementos: o subdiretório `profile/` contendo a imagem de identificação do paciente e subdiretórios para diferentes tipos de prontuários relacionados ao mesmo.

Os subdiretórios podem variar de acordo com o ciclo hospitalar do paciente, como:

- `consultas/`

- `exames/`

- `tratamentos/`

- `cirurgias/`

- `laboratoriais/`

- etc...

---

## 2 - Permissões

### 🔴 Baixo Nível

[placeholder]

### 🟡 Abstração do SO

[placeholder]

### 🟢 Abstração Programática (projeto)

[placeholder]

## 3. Mecanismo de Acesso

### 🔴 Baixo Nível

Chamadas de E/S passam por buffer cache, scheduler de I/O e drivers; mapeamento de blocos/setores e interrupções coordenam a leitura/escrita física.

[placeholder]

### 🟡 Abstração do SO

Syscalls open/read/write/close, descritores de arquivo e resolução de caminho via VFS padronizam o acesso e isolam a aplicação dos detalhes físicos.

Os mecanismos de acesso definem como os processos interagem com arquivos, abstraindo a complexidade do armazenamento físico através de interfaces padronizadas.

Nesta camada, descritores de arquivo, caminhos e chamadas de sistema (open, read, write, close) formam um CONTRATO estável que nosso trabalho reutiliza para oferecer operações de alto nível (listar, buscar, mapear) sem expor detalhes de baixo nível.

#### 🚧 Pontos fracos do acesso manual de arquivos

-  **Escalabilidade limitada:** Com o crescimento do volume de arquivos, navegar manualmente pela estrutura de diretórios torna-se cada vez mais demorado e propenso a erros

- **Busca ineficiente:** Localizar um prontuário específico requer conhecer exatamente sua localização na hierarquia de pastas, sem suporte para busca por conteúdo ou metadados

- **Inconsistência de nomenclatura:** Sem padronização automatizada, diferentes usuários podem criar arquivos com nomes inconsistentes, dificultando a organização

- **Dificuldade de integração:** Sistemas que dependem de acesso manual não se integram facilmente com outras ferramentas (relatórios, alertas, backup automatizado)

### 🟢 Abstração Programática (projeto)

A partir daqui, o conteúdo pertence à camada programática do projeto.

#### Abstraindo ainda mais o processo de acesso

Ao subir camadas de abstração, conseguimos mitigar os pontos fracos do gerenciamento manual de arquivos. Gerar a estrutura em um arquivo CSV adiciona uma camada estruturada de metadados que facilita consumo por outros sistemas.

Mapeamento de arquivos em formato tabular (CSV):

In [ ]:
# Listar todos os arquivos dos prontuários
save_and_display_csv("prontuarios", "mapeamento_prontuarios.csv")

Benefícios práticos do CSV gerado:


- **Visão unificada e filtrável:** status (ativo/arquivado), paciente, categoria, tipo, extensão, tamanho e caminho em um único lugar

- **Busca e filtros rápidos:** encontrar arquivos por extensão (.pdf, .jpg), por categoria (exames, consultas) ou por tamanho (arquivos grandes)

- **Integração fácil:** abre no Excel/Google Sheets, alimenta dashboards (BI) e scripts Python (Pandas)

- **Auditoria de mudanças:** comparar versões do CSV no controle de versão para ver adições/remoções e reorganizações de pastas

- **Automação e relatórios:** gerar KPIs (total por categoria, espaço ocupado por paciente, top N arquivos maiores)

- **Detecção de inconsistências:** nomes fora do padrão, arquivos em pastas erradas, extensões inesperadas

- **Portabilidade e compartilhamento:** compartilhar o mapa da estrutura sem expor o conteúdo sensível dos arquivos

- **Base para indexação:** serve como entrada para mecanismos de busca/indice sem varrer o disco toda hora

#### Mini relatório: quantidade de arquivos por categoria

Este gráfico mostra quantos arquivos existem em cada categoria (consultas, exames, etc.). Ele ilustra como o CSV permite gerar estatísticas rápidas sem navegar manualmente pelas pastas.

In [ ]:
gerar_relatorio_contagem_categoria()

---

## 4. Armazenamento

### 🔴 Baixo Nível

[placeholder]

### 🟡 Abstração do SO

[placeholder]

### 🟢 Abstração Programática (projeto)

[placeholder]